In [21]:
# !pip install libsvm
# !pip install -i https://pypi.douban.com/simple opencv-python
# !pip install tifffile
# !pip install -U libsvm-official
# !pip install -U libsvm-official
# !pip install torch
#!pip install scikit-learn



In [22]:
import torch
from libsvm.svmutil import *
import libsvm
import pandas as pd
import numpy as np
from PIL import Image
import cv2 as cv
import os
import torch
from torch.utils.data import random_split
import torch.utils.data as Data

# Libsvm


In [91]:

#遍历文件夹下文件名，，，依此读取每一张图片，，，转矩阵，，，降维，，，归一化，，，建立svm模型，，，输入到svm模型中,,,svmtrain,,,svmpredict
def get_fname(filepath):
    ffname = []
    for root,dirs,files in os.walk(filepath):
        for file in files:
            if(file.endswith("tif")):
                ffname.append(file)
    return(ffname)

# 方法一、PIL读取图片
def pil_img(pic1):
    img1 = Image.open(pic1)         #可以读取单通道影像,读取3通道16位tif影像时报错(PIL.UnidentifiedImageError: cannot identify image file),支持4通道8位影像
    arr = np.array(img1)
    arr = arr.reshape(1,-1)
    pix = (arr.shape)[1]
    return(arr)
#输入np.array ，在第零列插入标签 例如：女、男{1，2}
def add_label(tr1,label):
    a = [label]*len(tr1)
    tr1 = np.insert(tr1,0,values=a,axis=1)
    return (tr1)

#输入：文件夹，像素值大小，标签。   返回：np.array形式返回该文件夹中带标签的特征矩阵，n个图片*图片的像素
def get_t(filepath,pix,label):
    ts1= []
    ffname = get_fname(filepath)
    for i in ffname:
        ts1.append(pil_img(filepath+'\\'+i))
    ts1 = np.array(ts1)
    ts1 = ts1.reshape(len(ts1),pix)
    ts1 = ts1.astype(float)
    ts1 = add_label(ts1,label)
    ts1 = np.array(ts1)
    return(ts1)

#输入到模型前将图片特征矩阵中的值放缩，提高收敛速度，识别率
def cc2(train_y):
    for i in range(len(train_y)):
        for j in range(len(train_y[i])):
            train_y[i][j] = train_y[i][j]/255.0
    return (train_y)

def load_train(filepath1,filepath2,fold):
    if fold == "16_20":
        pixs = 320
    if fold == "48_60":
        pixs = 2880
    tr1 = get_t(filepath1,pixs,0)
    tr2 = get_t(filepath2,pixs,1)
    tr3 = np.vstack((tr1,tr2))
    train_x = tr3[:,0].tolist()
    train_y = tr3[:,1:].tolist()
    train_y = cc2(train_y)
    return train_x,train_y
def load_test(filepath3,filepath4,fold):
    if fold == "16_20":
        pixs = 320
    if fold == "48_60":
        pixs = 2880
    ts1 = get_t(filepath3,pixs,0)
    ts2 = get_t(filepath4,pixs,1)
    ts3 = np.vstack((ts1,ts2))
    test_x = ts3[:,0].tolist()
    test_y = ts3[:,1:].tolist()
    test_y = cc2(test_y)
    return test_x,test_y
def tt(train_x,train_y,test_x,test_y,val_x,val_y):
    
    
    options = ['-t 1 -g 1','-t 1 -g 2','-t 1 -g 3','-t 2 -c 1 -g 1 ','-t 2 -c 10 -g 1 ','-t 2 -c 100 -g 1 ','-t 2 -c 1 -g 0.1 ','-t 2 -c 10 -g 0.1 ','-t 2 -c 100 -g 0.1 ','-t 2 -c 1 -g 0.01 ','-t 2 -c 10 -g 0.01 ','-t 2 -c 100 -g 0.01 ','-t 2 -c 1000 -g 1 ','-t 2 -c 1000 -g 0.1 ','-t 2 -c 1000 -g 0.01 ']

    maxtrac = []
    maxvaac = []
    for i in options:
        svm_option = i

        model = svm_train(train_x,train_y,svm_option)
        print(f"LIBSBM中设置svm_option为<{svm_option}>训练集在测试集的预测结果为：")
        pre_labels, pre_acc, pre_vals = svm_predict(test_x,test_y,model)
        print(f"LIBSBM中设置svm_option为<{svm_option}>训练集在验证集的预测结果为：")
        pre_labels, pre_acc, pre_vals = svm_predict(val_x,val_y,model)
        
        print('*'*50)
        
def libsvmMethod():
    #训练集
    fold1,fold2 = "16_20","48_60"
    trfold1,trfold2,trfold3 = "tr_01","tr_02","tr_03"
    tsfold1,tsfold2,tsfold3 = "ts_01","ts_02","ts_03"
    valfold1,valfold2,valfold3 = "val_01","val_02","val_03"
    usingFold = fold2
    usingtrfold = trfold1
    usingtsfold = tsfold1
    usingvalfold = valfold1
    print(f"在{usingFold}的图片中，使用中的训练集为{usingtrfold}，使用的测试集为{usingtsfold}，使用的验证集为{usingvalfold}")
    filepath1 = fr"C:\Users\19925\Desktop\1\pdfbook\机器学习\机器学习（上饶）课程编程作业\GenderData\genderdata\{usingFold}\{usingtrfold}\Female"
    filepath2 = fr"C:\Users\19925\Desktop\1\pdfbook\机器学习\机器学习（上饶）课程编程作业\GenderData\genderdata\{usingFold}\{usingtrfold}\Male"
    #测试集
    filepath3 = fr"C:\Users\19925\Desktop\1\pdfbook\机器学习\机器学习（上饶）课程编程作业\GenderData\genderdata\{usingFold}\{usingtsfold}\Female"
    filepath4 = fr"C:\Users\19925\Desktop\1\pdfbook\机器学习\机器学习（上饶）课程编程作业\GenderData\genderdata\{usingFold}\{usingtsfold}\Male"
    #验证集
    filepath5 = fr"C:\Users\19925\Desktop\1\pdfbook\机器学习\机器学习（上饶）课程编程作业\GenderData\genderdata\{usingFold}\{usingvalfold}\Female"
    filepath6 = fr"C:\Users\19925\Desktop\1\pdfbook\机器学习\机器学习（上饶）课程编程作业\GenderData\genderdata\{usingFold}\{usingvalfold}\Male"
    # filepath1 = fr"C:\Users\ruijie\Desktop\yzj\pdfBook\112机器学习\机器学习（上饶）课程编程作业\GenderData\genderdata\16_20\tr_01\FemaleMale"
    # filepath2 = fr"C:\Users\ruijie\Desktop\yzj\pdfBook\112机器学习\机器学习（上饶）课程编程作业\GenderData\genderdata\16_20\tr_01\Male"
    
    
    train_x,train_y = load_train(filepath1,filepath2,usingFold)
    test_x,test_y =load_test(filepath3,filepath4,usingFold)
    val_x,val_y = load_test(filepath5,filepath6,usingFold)
    # 两种方式，tt（）运行参数集 ，注释即用
#     svm_option = '-t 2 -g 0.00288 -c 1000'
#     model = svm_train(train_x,train_y,svm_option)
#     print(f"LIBSBM中设置svm_option为{svm_option}训练集在测试集的预测结果为：")
#     pre_labels, pre_acc, pre_vals = svm_predict(test_x,test_y,model)
#     print(f"LIBSBM中设置svm_option为{svm_option}训练集在验证集的预测结果为：")
#     pre_labels, pre_acc, pre_vals = svm_predict(val_x,val_y,model)

    tt(train_x,train_y,test_x,test_y,val_x,val_y)
# libsvmMethod()

In [24]:
                # Experiment 1: Apply Support Vector Machines (SVMs) for gender classification. You will be
                # using the LibSVM implementation. Experiment both with polynomial and RBF kernels as well as
                # different C values. For consistency, try d=1, 2, and 3 for the polynomial kernel (note that
                # LibSVM provides two extra parameters for the polynomial kernel; to be consistent with the
                # lecture, set γ=1 and c0=0). In the case of the RBF kernel, try σ=1, 10, and 100. For the C value,
                # try C=1, 10, 100, and 1,000. Report your best results both for the 16x20 and 48x60 datasets.
# main()

# 贝叶斯
        # Model the male and female classes using a Gaussian distribution and use ML estimation to
        # estimate the parameters for each class. 

In [84]:
from sklearn import preprocessing

from sklearn.naive_bayes import GaussianNB

from sklearn.decomposition import PCA

def bayesMethod():
    #训练集
    fold1,fold2 = "16_20","48_60"
    trfold1,trfold2,trfold3 = "tr_01","tr_02","tr_03"
    tsfold1,tsfold2,tsfold3 = "ts_01","ts_02","ts_03"
    valfold1,valfold2,valfold3 = "val_01","val_02","val_03"
    usingFold = fold2
    usingtrfold = trfold1
    usingtsfold = tsfold1
    usingvalfold = valfold1
    print(f"在{usingFold}的图片中，使用中的训练集为{usingtrfold}，使用的测试集为{usingtsfold}，使用的验证集为{usingvalfold}")
    filepath1 = fr"C:\Users\19925\Desktop\1\pdfbook\机器学习\机器学习（上饶）课程编程作业\GenderData\genderdata\{usingFold}\{usingtrfold}\Female"
    filepath2 = fr"C:\Users\19925\Desktop\1\pdfbook\机器学习\机器学习（上饶）课程编程作业\GenderData\genderdata\{usingFold}\{usingtrfold}\Male"
    #测试集
    filepath3 = fr"C:\Users\19925\Desktop\1\pdfbook\机器学习\机器学习（上饶）课程编程作业\GenderData\genderdata\{usingFold}\{usingtsfold}\Female"
    filepath4 = fr"C:\Users\19925\Desktop\1\pdfbook\机器学习\机器学习（上饶）课程编程作业\GenderData\genderdata\{usingFold}\{usingtsfold}\Male"
    #验证集
    filepath5 = fr"C:\Users\19925\Desktop\1\pdfbook\机器学习\机器学习（上饶）课程编程作业\GenderData\genderdata\{usingFold}\{usingvalfold}\Female"
    filepath6 = fr"C:\Users\19925\Desktop\1\pdfbook\机器学习\机器学习（上饶）课程编程作业\GenderData\genderdata\{usingFold}\{usingvalfold}\Male"
    # filepath1 = fr"C:\Users\ruijie\Desktop\yzj\pdfBook\112机器学习\机器学习（上饶）课程编程作业\GenderData\genderdata\16_20\tr_01\FemaleMale"
    # filepath2 = fr"C:\Users\ruijie\Desktop\yzj\pdfBook\112机器学习\机器学习（上饶）课程编程作业\GenderData\genderdata\16_20\tr_01\Male"
    
    
    tr_y,tr_x = load_train(filepath1,filepath2,usingFold)
    ts_y,ts_x =load_test(filepath3,filepath4,usingFold)
    val_y,val_x = load_test(filepath5,filepath6,usingFold)
    # print(len(tr_x),len(tr_x[0]))
    pca = PCA(n_components=30)
    pca.fit(tr_x)
    # print(pca.explained_variance_ratio_) #降维后每一列方差占比
    # print(pca.explained_variance_) #降维后每一列方差
    # print(pca.n_components_) #降维后数据维度
    tr_pcax = pca.transform(tr_x).tolist()
    # print(type(tr_pcax))
    # print("trpca len is ",len(tr_pcax),len(tr_pcax[0]))
    print()
    gnb = GaussianNB()
    gnb.fit(tr_pcax,tr_y)
    ts_pcax = PCA(n_components=30)
    ts_pcax = pca.transform(ts_x).tolist()
    ts_score  = gnb.score(ts_pcax, ts_y)
    print(f"测试集分数：{ts_score}" )
    val_pcax = PCA(n_components=30)
    val_pcax = pca.transform(val_x).tolist()
    val_score  = gnb.score(val_pcax, val_y)
    print(f"测试集分数：{val_score}" )
    


# bayesMethod()

In [92]:
libsvmMethod()

在48_60的图片中，使用中的训练集为tr_01，使用的测试集为ts_01，使用的验证集为val_01
LIBSBM中设置svm_option为<-t 1 -g 1>训练集在测试集的预测结果为：
Accuracy = 90.9774% (121/133) (classification)
LIBSBM中设置svm_option为<-t 1 -g 1>训练集在验证集的预测结果为：
Accuracy = 92.4812% (123/133) (classification)
**************************************************
LIBSBM中设置svm_option为<-t 1 -g 2>训练集在测试集的预测结果为：
Accuracy = 90.9774% (121/133) (classification)
LIBSBM中设置svm_option为<-t 1 -g 2>训练集在验证集的预测结果为：
Accuracy = 92.4812% (123/133) (classification)
**************************************************
LIBSBM中设置svm_option为<-t 1 -g 3>训练集在测试集的预测结果为：
Accuracy = 90.9774% (121/133) (classification)
LIBSBM中设置svm_option为<-t 1 -g 3>训练集在验证集的预测结果为：
Accuracy = 92.4812% (123/133) (classification)
**************************************************
LIBSBM中设置svm_option为<-t 2 -c 1 -g 1 >训练集在测试集的预测结果为：
Accuracy = 43.609% (58/133) (classification)
LIBSBM中设置svm_option为<-t 2 -c 1 -g 1 >训练集在验证集的预测结果为：
Accuracy = 54.8872% (73/133) (classification)
******************************************

In [85]:
bayesMethod()

在48_60的图片中，使用中的训练集为tr_01，使用的测试集为ts_01，使用的验证集为val_01

测试集分数：0.8195488721804511
测试集分数：0.8721804511278195
